In [1]:
import os
import torch
import json
from io import BytesIO
import numpy as np
import torch.nn as nn
from tqdm import tqdm, trange
from model import ConvLSTM
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from utils import Dataset
from torch.utils.data.dataloader import DataLoader

In [2]:
device = torch.device('cpu')

In [3]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/trash'
result_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/ConvLSTM/psi_web_rho_150_last_1_layer_128_hidden'
cfg_path = os.path.join(result_path, 'cfg.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)
subdir_paths = sorted(os.listdir(data_path))
cr_paths = [os.path.join(data_path, p) for p in subdir_paths if p.startswith("cr")]

In [4]:
starting_slice = 0

dataset = Dataset(
    cr_paths=cr_paths,
    teacher_forcing=False,
    starting_slice=starting_slice,
    # v_min=cfg["v_min"],
    # v_max=cfg["v_max"],
    rho_min=cfg["rho_min"],
    rho_max=cfg["rho_max"],
)

In [5]:
model = ConvLSTM(
    input_dim=1,
    hidden_dim=cfg['hidden_dim'],
    kernel_size=(3, 3),
    num_layers=cfg['num_layers'],
    batch_first=True,
    bias=True,
    return_all_layers=False,
    output_dim=1
)

In [6]:
state_path = os.path.join(result_path, '20.pth')
model.load_state_dict(torch.load(state_path, map_location='cpu'))

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_4695/2926965003.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(state_path, m

<All keys matched successfully>

# Validation partition

In [7]:
i = 0
x, y = dataset[i]
print(x.shape, y.shape)
with torch.no_grad():
    # slicing
    # print(x.shape)
    x = torch.tensor(x, dtype=torch.float32)
    yhat, _ = model(x.unsqueeze(0), teacher_forcing=False, seq_len=y.shape[0])
    yhat = yhat.squeeze(0)
    print(yhat.shape)

(1, 1, 128, 128) (140, 1, 128, 128)
torch.Size([140, 1, 128, 128])


In [8]:
yhat.shape, y.shape

(torch.Size([140, 1, 128, 128]), (140, 1, 128, 128))

In [9]:
frames = []
for step in trange(yhat.shape[0]):
    val = y[step, 0, :, :]
    pred = yhat[step, 0, :, :]
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # plt.imshow(val, cmap="viridis")
    # # plt.contourf(
    # #     np.arange(val.shape[0]),
    # #     np.arange(val.shape[1]),
    # #     val.T,
    # #     cmap="viridis",
    # #     # norm=norm,
    # # )
    # plt.title(f"Actual Slice (k={step+1+starting_slice})")
    # plt.colorbar()
    # # plt.xlabel("I")
    # # plt.ylabel("J")

    # plt.subplot(1, 2, 2)
    # plt.imshow(pred, cmap="viridis")
    # # plt.contourf(
    # #     np.arange(pred.shape[0]),
    # #     np.arange(pred.shape[1]),
    # #     pred.T,
    # #     cmap="viridis",
    # #     # norm=norm,
    # # )
    # plt.title(f"Predicted Slice (k={step+1+starting_slice})")
    # plt.colorbar()
    # # plt.xlabel("I")
    # # plt.ylabel("J")
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    cmap = "viridis"

    # Plot first subplot
    im1 = axes[0].imshow(val, cmap=cmap)
    axes[0].set_title(f"Actual Slice (k={step+1+starting_slice})")

    # Plot second subplot
    im2 = axes[1].imshow(pred, cmap=cmap)
    axes[1].set_title(f"Predicted Slice (k={step+1+starting_slice})")

    # Create a single colorbar
    cbar = fig.colorbar(im1, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    # plt.show()

    # plt.show()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 140/140 [00:15<00:00,  8.90it/s]


In [10]:
output_filename = f"rho_convlstm.mp4"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps) as writer:
    for frame in frames:
        writer.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 600) to (1200, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [24]:
frames = []
for step in trange(yhat.shape[0]):
    val = y[step, 1, :, :]
    pred = yhat[step, 1, :, :]
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # plt.imshow(val, cmap="viridis")
    # # plt.contourf(
    # #     np.arange(val.shape[0]),
    # #     np.arange(val.shape[1]),
    # #     val.T,
    # #     cmap="viridis",
    # #     # norm=norm,
    # # )
    # plt.title(f"Actual Slice (k={step+1+starting_slice})")
    # plt.colorbar()
    # # plt.xlabel("I")
    # # plt.ylabel("J")

    # plt.subplot(1, 2, 2)
    # plt.imshow(pred, cmap="viridis")
    # # plt.contourf(
    # #     np.arange(pred.shape[0]),
    # #     np.arange(pred.shape[1]),
    # #     pred.T,
    # #     cmap="viridis",
    # #     # norm=norm,
    # # )
    # plt.title(f"Predicted Slice (k={step+1+starting_slice})")
    # plt.colorbar()
    # # plt.xlabel("I")
    # # plt.ylabel("J")
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    cmap = "viridis"

    # Plot first subplot
    im1 = axes[0].imshow(val, cmap=cmap)
    axes[0].set_title(f"Actual Slice (k={step+1+starting_slice})")

    # Plot second subplot
    im2 = axes[1].imshow(pred, cmap=cmap)
    axes[1].set_title(f"Predicted Slice (k={step+1+starting_slice})")

    # Create a single colorbar
    cbar = fig.colorbar(im1, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    # plt.show()

    # plt.show()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 140/140 [00:16<00:00,  8.63it/s]


In [25]:
output_filename = f"rho_convlstm.mp4"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps) as writer:
    for frame in frames:
        writer.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 600) to (1200, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
